In [ ]:
# =============================
# 1. Push Records from S3 to Kinesis
# =============================
import boto3, json, uuid, time
import pandas as pd
from datetime import datetime, timedelta

s3 = boto3.client('s3')
kinesis = boto3.client('kinesis')

BUCKET = 'fraud-model-artifacts'
KEY = 'creditcard.csv'
STREAM = 'creditcard-stream'

# Read and sample data
obj = s3.get_object(Bucket=BUCKET, Key=KEY)
df = pd.read_csv(obj['Body'])
df = df.sample(10).copy()  # for quick test

In [ ]:
df = df.drop(columns=['Class', 'Time', 'event_time'], errors='ignore')

In [ ]:
# Send to Kinesis
for i, (_, row) in enumerate(df.iterrows()):
    record = row.to_dict()
    kinesis.put_record(
        StreamName=STREAM,
        Data=json.dumps(record),
        PartitionKey='partition-1'
    )
    print(f"Sent record {i}")
    time.sleep(0.1)

In [ ]:
#minimal kinesis consumer
client = boto3.client('kinesis', region_name='us-east-1')
stream = 'creditcard-stream'

# Get the shard ID
shard_id = client.describe_stream(StreamName=stream)['StreamDescription']['Shards'][0]['ShardId']

# Get iterator to start reading
shard_it = client.get_shard_iterator(
    StreamName=stream,
    ShardId=shard_id,
    ShardIteratorType='TRIM_HORIZON'  # from earliest #LATEST #TRIM_HORIZON
)['ShardIterator']

# Read records
records = client.get_records(ShardIterator=shard_it, Limit=100)
i=0
for r in records['Records']:
    i+=1
    #print(json.loads(r['Data']))
print(i)